## Documentação parquet particionado

https://spark.apache.org/docs/latest/sql-data-sources-parquet.html 

O formato Parquet é amplamente utilizado para armazenamento de dados em ambientes distribuídos devido à sua eficiência de compressão e leitura. Uma das principais vantagens do Parquet é o suporte nativo à partição de dados, permitindo organizar grandes volumes de informações em diretórios particionados por colunas, como datas ou categorias. Isso facilita consultas mais rápidas e eficientes, pois apenas as partições relevantes são lidas durante a análise. Nesta seção, exploraremos como criar, ler e manipular arquivos Parquet particionados utilizando o Apache Spark.

In [0]:
path = "/Volumes/workspace/default/tutorial/anac"

In [0]:
dfpq = spark.read.parquet(f"{path}/V_OCORRENCIA_AMPLA_PARQUET")
display(dfpq)

In [0]:
# Verificando dados distintos de colunas com spark sql
display(dfpq.select("Classificacao_da_Ocorrência").distinct())


In [0]:
dfpq.write \
    .partitionBy("Classificacao_da_Ocorrência") \
    .mode("overwrite") \
    .parquet(f"{path}/V_OCORRENCIA_AMPLA_PARQUET_PARTICIONED")

In [0]:
display(dbutils.fs.ls(path))

###Vendo Arquivos particionados

O Apache Spark permite trabalhar com arquivos Parquet particionados, que são organizados em subdiretórios conforme valores de uma ou mais colunas. Isso facilita a consulta eficiente de grandes volumes de dados, pois apenas as partições relevantes são lidas durante a análise. Nesta seção, veremos como visualizar e explorar a estrutura desses arquivos particionados.

In [0]:
display(dbutils.fs.ls(f"{path}/V_OCORRENCIA_AMPLA_PARQUET_PARTICIONED"))

In [0]:
# Descendo mais um nivel ver os logs de cada partição
display(dbutils.fs.ls(f"{path}/V_OCORRENCIA_AMPLA_PARQUET_PARTICIONED/Classificacao_da_Ocorrência=Acidente/"))


In [0]:
dfacidente = spark.read.parquet(f"{path}/V_OCORRENCIA_AMPLA_PARQUET_PARTICIONED/Classificacao_da_Ocorrência=Ocorrência de Solo/")
display(dfacidente)

### Ler todos os dados mesmo particionados (agrupados)

Ao trabalhar com arquivos Parquet particionados, é comum que os dados estejam organizados em subdiretórios conforme valores de uma ou mais colunas. No entanto, em algumas situações, pode ser necessário ler todos os dados disponíveis, independentemente das partições existentes. Nesta seção, veremos como o Apache Spark permite acessar e processar o conjunto completo de dados, mesmo quando eles estão distribuídos em múltiplas partições, garantindo flexibilidade e eficiência na análise.

In [0]:
dfTudo = spark.read\
    .format("parquet")\
    .load(f"{path}/V_OCORRENCIA_AMPLA_PARQUET_PARTICIONED")

display(dfTudo)

###Salvando em mais de 1 partição

Ao salvar dados em formato Parquet utilizando o Apache Spark, é possível particioná-los em múltiplas colunas, como data e categoria, criando uma estrutura de diretórios aninhados. Essa abordagem permite organizar grandes volumes de dados de forma eficiente, otimizando consultas e facilitando a manutenção. Nesta seção, veremos como salvar um DataFrame em mais de uma partição, aproveitando os benefícios do particionamento múltiplo para análises mais rápidas e flexíveis.

In [0]:
# Uma equipe de cada estado analisa as "Classificacao da Ocorrência" e cada uma das ocorrencias é analizado por uma pessoa.
# Separarar (particionar por Classificacao da Ocorrência e estado)


dfpq.write \
    .partitionBy("UF", "Classificacao_da_Ocorrência") \
    .mode("overwrite") \
    .parquet(f"{path}/V_OCORRENCIA_AMPLA_PARQUET_MULTIPARTICIONED")

#Obs: pode demorar na escrita devido aos particionamentos , mas ganha tempo na leitura

In [0]:
display(dbutils.fs.ls(path))

In [0]:
display(dbutils.fs.ls(f"{path}/V_OCORRENCIA_AMPLA_PARQUET_MULTIPARTICIONED"))

In [0]:
display(dbutils.fs.ls(f"{path}/V_OCORRENCIA_AMPLA_PARQUET_MULTIPARTICIONED/UF=MG/"))

In [0]:
display(dbutils.fs.ls(f"{path}/V_OCORRENCIA_AMPLA_PARQUET_MULTIPARTICIONED/UF=MG/Classificacao_da_Ocorrência=Acidente/"))

In [0]:
# Simulando o responsavel de Mg onde a Classificacao da Ocorrência seja igual a Acidente.
dfmg = spark.read \
    .format("parquet") \
    .load(f"{path}/V_OCORRENCIA_AMPLA_PARQUET_MULTIPARTICIONED/UF=MG/Classificacao_da_Ocorrência=Acidente/")

display(dfmg)